In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

In [27]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.8 MB/s eta 0:00:00a 0:00:01
--- Logging error ---
Traceback (most recent call last):
  File "/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Users/kevinshang/opt

In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

**Load the tables**

In [9]:
lf = pd.read_csv('LF_train.csv')
lh = pd.read_csv('LH_train.csv')
rf = pd.read_csv('RF_train.csv')
rh = pd.read_csv('RH_train.csv')

**Clean up NANs (techniques: get rid of the rows that contain NANs)**

In [10]:
lf = lf.dropna()
lh = lh.dropna()
rf = rf.dropna()
rh = rh.dropna()

In [23]:
rf

,id,gender,weight,age,RF,speed,V1_LF,V1_RF,V1_LH,V1_RH,...,V28_LH_trot,V28_RH_trot,V29_LF_trot,V29_RF_trot,V29_LH_trot,V29_RH_trot,V30_LF_trot,V30_RF_trot,V30_LH_trot,V30_RH_trot
0,124,1,26.500,4.287671,0,0.978,167.969885,171.888387,102.337537,98.746290,...,0.217356,0.222414,0.272885,0.271068,0.229623,0.226424,0.294851,0.289220,0.207965,0.207965
1,17,0,31.100,3.895890,1,1,231.120285,219.015361,150.547964,148.561574,...,0.217737,0.209934,0.328321,0.267056,0.202836,0.201787,0.273036,0.280310,0.220660,0.225994
3,4,0,57.154,6.906849,0,1.05,410.410023,441.481866,252.853942,286.218337,...,0.196136,0.199127,0.295163,0.265333,0.213271,0.226234,0.263292,0.268396,0.228413,0.239898
4,112,1,37.200,1.378082,1,1.12,199.454162,204.450412,129.766989,112.402628,...,0.227054,0.214635,0.293653,0.288617,0.218351,0.199379,0.290065,0.290428,0.210297,0.209210
5,46,0,28.500,4.090411,1,1.048333333,193.208201,201.830751,105.614619,115.080034,...,0.181027,0.170701,0.335157,0.301296,0.187485,0.176063,0.265664,0.287594,0.209900,0.236842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,79,1,31.700,4.693151,0,1.123333333,206.314242,205.633455,146.737645,121.406073,...,0.216436,0.171680,0.313494,0.287276,0.223702,0.175527,0.278817,0.280024,0.232348,0.208811
101,145,0,30.900,2.728767,0,1.138,211.611292,197.700189,125.636298,114.337032,...,0.248401,0.237643,0.231912,0.257847,0.252546,0.257694,0.282596,0.287447,0.212856,0.217101
104,64,1,38.200,2.205479,1,1.13,206.373586,225.870360,159.504339,152.115178,...,0.207917,0.214148,0.241561,0.321349,0.207127,0.229963,0.272984,0.263083,0.235266,0.228666
106,134,0,24.500,9.249315,0,1.094,161.125594,167.335178,118.231750,118.413977,...,0.237425,0.251038,0.245745,0.253815,0.244468,0.255972,0.286004,0.278567,0.216362,0.219067


**Train Test Split**

In [14]:
xTr, xTe, yTr, yTe = train_test_split(lh.drop('LH', axis = 1), lh['LH'], test_size=0.3, random_state=42)
tableX = lh.drop('LH', axis = 1)

In [15]:
# Create base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)

# Create Adaboost classifier
clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

# Train classifier
clf.fit(xTr, yTr)

# Make predictions on the testing dataset
y_pred = clf.predict(xTe)

# Calculate the macro F1 score on the testing dataset
f1_macro = f1_score(yTe, y_pred, average='macro')

# Print the macro F1 score
print('Testing macro F1 score: %0.3f' % f1_macro)

Testing macro F1 score: 0.902


In [28]:
xTr, xTe, yTr, yTe = train_test_split(lf.drop('LF', axis = 1), lf['LF'], test_size=0.3, random_state=42)
tableX = lf.drop('LF', axis = 1)

In [29]:
# Create base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)

# Create Adaboost classifier
clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

# Train classifier
clf.fit(xTr, yTr)

# Make predictions on the testing dataset
y_pred = clf.predict(xTe)

# Calculate the macro F1 score on the testing dataset
f1_macro = f1_score(yTe, y_pred, average='macro')

# Print the macro F1 score
print('Testing macro F1 score: %0.3f' % f1_macro)

Testing macro F1 score: 0.410


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Train a random forest classifier on the training data
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(xTr, yTr)

# Evaluate the classifier on the testing data
y_pred = rf.predict(xTe)

f1_macro = f1_score(yTe, y_pred, average='macro')

# Print the classification report and macro F1 score
print(classification_report(yTe, y_pred))
print(f"Macro F1 score: {f1_macro:.4f}")


              precision    recall  f1-score   support

           0       0.74      1.00      0.85        17
           1       0.00      0.00      0.00         6

    accuracy                           0.74        23
   macro avg       0.37      0.50      0.42        23
weighted avg       0.55      0.74      0.63        23

Macro F1 score: 0.4250


/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kevinshang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
import xgboost as xgb
from sklearn.metrics import classification_report

# Create an XGBoost classifier with the desired hyperparameters
params = {
    'objective': 'binary:logistic',  # replace with the desired objective function
    'max_depth': 3,  # replace with the desired maximum tree depth
    'learning_rate': 0.1,  # replace with the desired learning rate
    'n_estimators': 100  # replace with the desired number of trees
}
xgb_clf = xgb.XGBClassifier(**params)

# Train the classifier on the training dataset
xgb_clf.fit(xTr, yTr)

print("what")

# Make predictions on the testing dataset
y_pred = xgb_clf.predict(xTe)

macro_f1 = f1_score(yTe, y_pred, average='macro')

# Print the macro F1 score
print("Macro F1 score on testing data: {:.4f}".format(macro_f1))


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:speed: object, Speed: object

In [16]:
xTr, xTe, yTr, yTe = train_test_split(rh.drop('RH', axis = 1), rh['RH'], test_size=0.3, random_state=42)
tableX = rh.drop('RH', axis = 1)

In [17]:
# Create base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)

# Create Adaboost classifier
clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

# Train classifier
clf.fit(xTr, yTr)

# Make predictions on the testing dataset
y_pred = clf.predict(xTe)

# Calculate the macro F1 score on the testing dataset
f1_macro = f1_score(yTe, y_pred, average='macro')

# Print the macro F1 score
print('Testing macro F1 score: %0.3f' % f1_macro)

Testing macro F1 score: 0.817


In [38]:
xTr, xTe, yTr, yTe = train_test_split(rf.drop('RF', axis = 1), rf['RF'], test_size=0.3, random_state=42)
tableX = rf.drop('RF', axis = 1)

AttributeError: 'RandomForestClassifier' object has no attribute 'drop'

In [19]:
# Create base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)

# Create Adaboost classifier
clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

# Train classifier
clf.fit(xTr, yTr)

# Make predictions on the testing dataset
y_pred = clf.predict(xTe)

# Calculate the macro F1 score on the testing dataset
f1_macro = f1_score(yTe, y_pred, average='macro')

# Print the macro F1 score
print('Testing macro F1 score: %0.3f' % f1_macro)

Testing macro F1 score: 0.488


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Train a random forest classifier on the training data
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(xTr, yTr)

# Evaluate the classifier on the testing data
y_pred = rf.predict(xTe)

f1_macro = f1_score(yTe, y_pred, average='macro')

# Print the classification report and macro F1 score
print(classification_report(yTe, y_pred))
print(f"Macro F1 score: {f1_macro:.4f}")


**Concate data frames**

In [89]:
tempTable1 = pd.concat([lf, lh], axis=0)
tempTable2 = pd.concat([rf, rh], axis=0)
tempTable3 = pd.concat([tempTable1, tempTable2], axis=0)

tempTable3.head()

,id,dob,gender,weight,forceplate_date,age,LF,gait,speed,V1_LF,...,V29_RF_trot,V29_LH_trot,V29_RH_trot,V30_LF_trot,V30_RF_trot,V30_LH_trot,V30_RH_trot,LH,RF,RH
0,169,01/29/2008,0,60.0,2009-08-14,1.542466,0.0,Walk,0.9925,381.595253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35,2006-04-12 00:00:00,0,37.0,2011-07-19,5.271233,0.0,Walk,1.046,241.735368,...,0.253702,0.208299,0.281883,0.284936,0.290986,0.208711,0.215366,NaN,NaN,NaN
2,166,09/24/1999,1,24.1,2008-04-30,8.605479,0.0,Walk,1.152,183.448346,...,0.277115,0.226666,0.232730,0.282723,0.280479,0.212416,0.224383,NaN,NaN,NaN
3,41,2003-03-05 00:00:00,0,69.3,2010-06-22,7.304110,0.0,Walk,0.95,457.200158,...,0.313119,0.218540,0.228796,0.275581,0.279070,0.227326,0.218023,NaN,NaN,NaN
4,99,2002-06-27 00:00:00,1,26.7,2009-01-23,6.580822,0.0,Walk,1.014,174.265686,...,0.279860,0.259193,0.176733,0.297174,0.281677,0.223336,0.197812,NaN,NaN,NaN


**Clean Duplicates**

In [93]:
duplicates = tempTable3.duplicated(subset=["id"])

table = tempTable3[~duplicates]

table.head()

table.to_csv('cleanedTable.csv', index = False)

In [ ]:
prefix = "STD"
cols_to_keep = [col for col in df.columns if not col.startswith(prefix)]
df_filtered = df.loc[:, cols_to_keep]